In [1]:
# pip install tabulate faiss-cpu 

### Load DataBase

In [2]:
import requests

path = 'data.txt'
url = 'https://raw.githubusercontent.com/Aliraqimustafa/EasyShop-AI/main/data.txt'


r = requests.get(url)
with open(path,'wb') as f:
    f.write(r.content)

In [3]:
import pandas as pd
import numpy as np

df = pd.read_csv('data.txt').drop_duplicates()
df.head()

,Product name,Product price,Product description,Brand
0,Laptop,799.0,High-performance laptop with Intel Core i7 pro...,Dell
1,Smartwatch,249.0,"Fitness smartwatch with heart rate monitoring,...",Fitbit
2,Headphones,149.0,Noise-cancelling Bluetooth headphones with cri...,Bose
3,Camera,499.0,"DSLR camera with 24MP sensor, interchangeable ...",Canon
4,Printer,119.0,"Compact wireless printer with AirPrint, auto-d...",HP


In [4]:
df.shape

(164, 4)

### Search Engain
1. by name
2. by description
3. by price
4. Optional[by Brand]

In [5]:
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.docstore.document import Document

embeddings = OpenAIEmbeddings()

In [6]:
names = df['Product name'].apply(lambda x : x.lower()).values
descriptions = df['Product description'].apply(lambda x : x.lower()).values
prices = df['Product price'].values
brand = df['Brand'].apply(lambda x : x.lower()).values

### Convert Database to Document

In [7]:
def metadata(lst):
    n_list = list(range(len(lst)))
    price = []
    for i in n_list:
        _ = df.iloc[i]['Product price']
        price.append(_)
    return zip(lst , n_list , price)
def _Document(i):
    data = {
        'page_content' : i[0],
        'metadata' : {'n':i[1],'price':i[2]}
    }
    return Document(**data)

In [8]:
names_docs = [_Document(i) for i in metadata(names)]
names_docs[:3]

[Document(page_content='laptop', metadata={'n': 0, 'price': 799.0}),
 Document(page_content='smartwatch', metadata={'n': 1, 'price': 249.0}),
 Document(page_content='headphones', metadata={'n': 2, 'price': 149.0})]



In [9]:
descriptions_docs = [_Document(i) for i in metadata(descriptions)]
descriptions_docs[:3]

[Document(page_content='high-performance laptop with intel core i7 processor, 16gb ram, and 512gb ssd storage.', metadata={'n': 0, 'price': 799.0}),
 Document(page_content='fitness smartwatch with heart rate monitoring, gps tracking, and 7-day battery life.', metadata={'n': 1, 'price': 249.0}),
 Document(page_content='noise-cancelling bluetooth headphones with crisp sound quality and 30-hour playtime.', metadata={'n': 2, 'price': 149.0})]


### Crate faiss database

In [11]:
### Save [database, knowledgebase]

descriptions_db = FAISS.from_documents(documents = descriptions_docs ,embedding = embeddings )
descriptions_db.save_local(folder_path ='descriptions_db' , index_name= 'descriptions')
names_db = FAISS.from_documents(documents = names_docs ,embedding = embeddings )
names_db.save_local(folder_path ='names_db' , index_name= 'names')

In [12]:
### Load [database, knowledgebase]

descriptions_db = FAISS.load_local('descriptions_db',index_name= 'descriptions' , embeddings=embeddings)
names_db = FAISS.load_local('names_db',index_name= 'names' , embeddings=embeddings)

### Crate Functions to search in ` database or knowledgebase `

In [13]:
def search_name(name):
    reriever_docs = names_db.similarity_search(query = name, k=50)
    return reriever_docs

def search_description(description,reriever_docs_names):
    reriever_docs_names_n = [i.metadata['n']  for i in reriever_docs_names]
    
    reriever_docs = descriptions_db.similarity_search(query = description, k=50)
    
    indexes = []
    for i,j in enumerate(reriever_docs):
        if j.metadata['n'] in reriever_docs_names_n:
            indexes.append(i)
     
    reriever_docs2 = []    
    for i in indexes:
        reriever_docs2.append(reriever_docs[i])
    
    return reriever_docs2

def search_price(price,avrage,reriever_docs_description):
    n_list = [i.metadata['n']  for i in reriever_docs_description]
    price_list = [i.metadata['price']  for i in reriever_docs_description]
    
    
    indexes = []
    for j,i in enumerate(price_list):
        if    price-avrage<i and i<price+avrage:
            indexes.append(j)
    
    reriever_docs = []
    for i in indexes:
        reriever_docs.append(reriever_docs_description[i])
    
    return reriever_docs

def final_search(result_search_price):
    
    indexes = []
    for i in result_search_price:
        _ = i.metadata['n']
        indexes.append(_)
    
    results = []
    for i in indexes:
        _ = df.iloc[i:i+1]
        results.append(_)
    return pd.concat(results)

def Search(product_name , product_description , product_Price , product_Price_delta ):
    
    """
    product_name : Name of Product
    product_description : Description of Product
    product_Price : The price you can pay to buy the produ
    product_Price_delta : Value that can be added or subtracted from product_Price
    
    ًExample : 
        -----------------------------------
        1- product_name : Laptop
        2- product_description : laptop that's for gaming
        3- product_Price : 600
        4- product_Price_delta : 200
        -----------------------------------
        Step 1. : 
            1-  Search about 'Laptop' keyword in database
        Step 2. : 
            1-  Search about 'laptop that's for gaming' description in database
            2- Return results matching the description and product name
        Step 3. : 
            1-  Return the identical results between the name and description of the product as well as the price that the user can pay on this product
        Step 4. : 
            1- In the end, we return all the matched documents. 
        -----------------------------------
    """
    _search_name = search_name(product_name)
    _search_description = search_description(product_description , _search_name)
    _search_price = search_price(product_Price , product_Price_delta , _search_description)
    final_result = final_search(_search_price)
    return {'Documents' : final_result}

### Test Search without AI

In [23]:
Documents_final_result = Search(
    product_name = "laptop",
    product_description = "laptop that's for gaming",
    product_Price = 600,
    product_Price_delta = 200,)

Documents_final_result = Documents_final_result['Documents']
Documents_final_result.head()

,Product name,Product price,Product description,Brand
0,Laptop,799.0,High-performance laptop with Intel Core i7 pro...,Dell
20,Gaming Console,699.0,"Cutting edge gaming console with ray tracing, ...",Microsoft
195,Tablet,629.0,Detachable 2-in-1 tablet with 3000 x 2000 IPS ...,Microsoft Surface Pro 7
162,Tablet,629.0,Detachable 2-in-1 tablet with 3000 x 2000 IPS ...,Microsoft Surface Pro 7
26,Gaming Console,799.0,Ultra high speed gaming console with up to 8K ...,Microsoft


### Search with AI

In [24]:
import json
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferMemory
from langchain.agents import AgentType, initialize_agent
from langchain.chains import LLMChain
from langchain.tools import Tool
from langchain import PromptTemplate
llm = OpenAI(temperature=.0)
memory = ConversationBufferMemory(memory_key='chat_history')
agent_type = AgentType.CONVERSATIONAL_REACT_DESCRIPTION

In [25]:
template = \
'''\
System : You are an ai assistant you should Extract the following key from Human message :
    [product_name , product_description , product_Price , product_Price_delta]
    and return a dict as json file.
    Example : 
        query = Hello, I would like to buy a laptop with powerful specifications for gaming at a price between 600 and 900
        -----------------------------------
        result to search :
        -----------------------------------
        1- product_name : Laptop
        2- product_description : laptop that's for gaming
        3- product_Price : 600
        4- product_Price_delta : 200
        -----------------------------------
        Step 1. : 
            1-  Search about 'Laptop' keyword in database
        Step 2. : 
            1-  Search about 'laptop that's for gaming' description in database
            2- Return results matching the description and product name
        Step 3. : 
            1-  Return the identical results between the name and description of the product as well as the price that the user can pay on this product
        Step 4. : 
            1- In the end, we return all the matched documents. 
        -----------------------------------
Human message: {inputs}

json file:'''

In [26]:
prompt_template = PromptTemplate(input_variables=['inputs'] ,template=template )
llm_chain = LLMChain(llm=llm, verbose=False, prompt=prompt_template)

In [27]:
def Search_with_llm(text):
    _ = llm_chain.run(inputs = text)
    _2 = json.loads(_)
    ## '_Search' is DataFrame
    _Search = Search(**_2)['Documents']
    index = list(_Search.index)
    txt = 'Source is df.iloc[{n}]'
    Source = [txt.format(n=n) for n in index]
    _produc_name = _Search['Product name']
    _produc_price = _Search['Product price']
    _produc_description = _Search['Product description']
    _produc_Brand = _Search['Brand']
    
    docs = zip(Source,_produc_name,_produc_price,_produc_description,_produc_Brand)
    return list(docs) 

In [29]:
_1 = llm_chain.run(inputs = 'Hello, I would like to buy a laptop with powerful specifications for gaming at a price between 600 and 900')


In [28]:
print(_1)
print(type(_1))

{
    "product_name": "Laptop",
    "product_description": "laptop that's for gaming",
    "product_Price": 600,
    "product_Price_delta": 200
}
<class 'str'>


In [33]:
_json = json.loads(_1)

In [34]:
print(_json)
print(type(_json))

{'product_name': 'Laptop', 'product_description': "laptop that's for gaming", 'product_Price': 600, 'product_Price_delta': 200}
<class 'dict'>


### Test Search with AI 

In [36]:
Docs_with_llm = \
Search_with_llm('Hello, I would like to buy a laptop with powerful specifications for gaming at a price between 600 and 900')

In [38]:
print(Docs_with_llm[:3])

[('Source is df.iloc[0]',
  'Laptop',
  799.0,
  'High-performance laptop with Intel Core i7 processor, 16GB RAM, and 512GB SSD storage.',
  'Dell'),
 ('Source is df.iloc[20]',
  'Gaming Console',
  699.0,
  'Cutting edge gaming console with ray tracing, 4K gaming at 120 fps, ultra high speed SSD.',
  'Microsoft'),
 ('Source is df.iloc[195]',
  'Tablet',
  629.0,
  'Detachable 2-in-1 tablet with 3000 x 2000 IPS display,Intel Core i5 CPU, keyboard with touchpad.',
  'Microsoft Surface Pro 7')]


In [39]:
len(Docs_with_llm)

15

### Final Search with AI `EsayShop AI Agent`

In [40]:
tools = [
    Tool.from_function(
        func=Search_with_llm,
        name="Search_any_products",
        description="Useful when you want to answer questions about any products."
    ),
]

In [41]:
agent = initialize_agent(agent=agent_type, 
                         tools=tools, 
                         verbose=True,
#                          memory=memory,
                        llm =llm )

In [42]:
agent.agent.llm_chain.prompt.input_variables

['input', 'chat_history', 'agent_scratchpad']

In [43]:
print(agent.agent.llm_chain.prompt.template)

Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful tool that can help with a wide range of tasks 

In [44]:
# if you comment memory in agent you should remove memory_key[chat_history] 
# from 'agent.agent.llm_chain.prompt.template' and agent.agent.llm_chain.prompt.input_variables
# i have comment memory so i will remove memory_key from template and input_variables

In [45]:
agent.agent.llm_chain.prompt.input_variables = ['input',  'agent_scratchpad']


In [46]:
agent.agent.llm_chain.prompt.template = \
'''
System: If you use one of the tools, you must answer the person in a respectful manner and give him the source of the product, for example, the name of the product, product specifications, and brand. You should suggest some similar products to him in a way that will make the person actually buy. Give him a detailed answer as if you were the person's personal companion and helped him buy the products. There must be a blank line between each product and another that you submit. Do not forget to provide the source of the product, for example: df.iloc[10]


Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful tool that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.

TOOLS:
------

Assistant has access to the following tools:

> Search_any_products: Useful when you want to answer questions about any products.

To use a tool, please use the following format:

```
Thought: Do I need to use a tool? Yes
Action: the action to take, should be one of [Search_any_products]
Action Input: the input to the action
Observation: the result of the action
```

When you have a response to say to the Human, or if you do not need to use a tool, you MUST use the format:

```
Thought: Do I need to use a tool? No
AI: [your response here]
```

Begin!


New input: {input}
{agent_scratchpad}
'''

### Test Agent

In [48]:
_2 = agent.run("Hello, I would like to buy a laptop with powerful specifications for gaming at a price between 600 and 900")


> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: Search_any_products
Action Input: Laptop with powerful specifications for gaming at a price between 600 and 900
Observation: [('Source is df.iloc[7]', 'Laptop', 849.0, 'High-performance laptop with Intel Core i7 processor, 16GB RAM, and 1TB SSD storage.', 'Asus'), ('Source is df.iloc[0]', 'Laptop', 799.0, 'High-performance laptop with Intel Core i7 processor, 16GB RAM, and 512GB SSD storage.', 'Dell'), ('Source is df.iloc[14]', 'Laptop', 899.0, 'High-performance laptop with Intel Core i9 processor, 32GB RAM, 1TB SSD storage.', 'MSI'), ('Source is df.iloc[195]', 'Tablet', 629.0, 'Detachable 2-in-1 tablet with 3000 x 2000 IPS display,Intel Core i5 CPU, keyboard with touchpad.', 'Microsoft Surface Pro 7'), ('Source is df.iloc[162]', 'Tablet', 629.0, 'Detachable 2-in-1 tablet with 3000 x 2000 IPS display,Intel Core i5 CPU, keyboard with touchpad.', 'Microsoft Surface Pro 7 '), ('Source is df.iloc[20]', 'Ga

In [49]:
print(_2)

Based on your request, I have found several products that may be suitable for you. 

The Asus laptop (Source: df.iloc[7]) is a high-performance laptop with an Intel Core i7 processor, 16GB RAM, and 1TB SSD storage, and is priced at 849. 

The Dell laptop (Source: df.iloc[0]) is a high-performance laptop with an Intel Core i7 processor, 16GB RAM, and 512GB SSD storage, and is priced at 799. 

The MSI laptop (Source: df.iloc[14]) is a high-performance laptop with an Intel Core i9 processor, 32GB RAM, 1TB SSD storage, and is priced at 899. 

The Microsoft Surface Pro 7 (Source: df.iloc[195]) is a detachable 2-in-1 tablet with a 3000 x 2000 IPS display, Intel Core i5 CPU, and a keyboard with touchpad, and is priced at 629. 

The Microsoft Surface Pro 7 (Source: df.iloc[162]) is a detachable 2-in-1 tablet with a 3000


In [50]:
df.iloc[7]

Product name                                                      Laptop
Product price                                                      849.0
Product description    High-performance laptop with Intel Core i7 pro...
Brand                                                               Asus
Name: 7, dtype: object